### Downloaded QIIME-formatted ref seqs from UNITE
### https://unite.ut.ee/repository.php
### When using this resource, please cite it as follows:
#### Abarenkov, Kessy; Zirk, Allan; Piirmann, Timo; Pöhönen, Raivo; Ivanov, Filipp; Nilsson, R. Henrik; Kõljalg, Urmas (2022): UNITE QIIME release for Fungi 2. Version 16.10.2022. UNITE Community. https://doi.org/10.15156/BIO/2483916
#### Includes global and 3% distance singletons. 

### Using the UNITE-recommended version of the database (not developer)

In [1]:
# Import UNITE db sequences
# Using the dynamic set, informed by mycologists
# The UNITE databse docs recommend using the version they have already trimmed.
!qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path UNITE_2022_10_16_Global_and_3pct_singletons/sh_refs_qiime_ver9_dynamic_s_29.11.2022.fasta \
  --output-path ../Seq-processing/unite.trimmed.qza

Imported UNITE_2022_10_16_Global_and_3pct_singletons/sh_refs_qiime_ver9_dynamic_s_29.11.2022.fasta as DNASequencesDirectoryFormat to ../Seq-processing/unite.trimmed.qza


In [2]:
# Import UNITE db taxonomy
# Using the dynamic set, informed by mycologists
# And the developer version, which is not as trimmed, as recommended in QIIME feature classifier tutorial
!qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path  UNITE_2022_10_16_Global_and_3pct_singletons/sh_taxonomy_qiime_ver9_dynamic_s_29.11.2022.txt \
  --output-path ../Seq-processing/unite-taxonomy.trimmed.qza

Imported UNITE_2022_10_16_Global_and_3pct_singletons/sh_taxonomy_qiime_ver9_dynamic_s_29.11.2022.txt as HeaderlessTSVTaxonomyFormat to ../Seq-processing/unite-taxonomy.trimmed.qza


In [3]:
# Train the classifier
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads ../Seq-processing/unite.trimmed.qza \
  --i-reference-taxonomy ../Seq-processing/unite-taxonomy.trimmed.qza \
  --o-classifier ../Seq-processing/unite.classifier.trimmed.qza

Saved TaxonomicClassifier to: ../Seq-processing/unite.classifier.trimmed.qza


Note: in previous tests, the UNITE-recommended dataset gave assignments to more OTUs at the genus and species levels in the 2015 dataset, and more classified OTUs in the 2019 dataset as well. Given this, I will use the trimmed version, since it is the recommended option from UNITE.

## Running both years without having run ITSxpress first
ITSx / ITSxpress improves taxonomic assignments, but I'm concerned that differential trimming
between the two years' datasets might artificially lead to different OTUs that should be the same.
If we keep the ends of the amplicon, that helps give confidence that it's not a weird trimming issue

In [3]:
# Since not running ITSx, do need to trim primers in case short amplicons got primers sequenced into at 5' end.
!qiime cutadapt trim-paired \
    --i-demultiplexed-sequences ../Seq-processing/sequences.qza \
    --p-cores 18 \
    --p-adapter-f AACTTTYRRCAAYGGATCWCT \
    --p-adapter-r AGCCTCCGCTTATTGATATGCTTAART \
    --p-error-rate 0.1 \
    --p-match-adapter-wildcards \
    --p-discard-untrimmed False \
    --o-trimmed-sequences ../Seq-processing/sequences.cutadapt.qza

Saved SampleData[PairedEndSequencesWithQuality] to: ../Seq-processing/sequences.cutadapt.qza


In [4]:
# Inspect efects
! qiime demux summarize \
  --i-data ../Seq-processing/sequences.cutadapt.qza \
  --o-visualization ../Seq-processing/sequences.cutadapt.qzv

Saved Visualization to: ../Seq-processing/sequences.cutadapt.qzv


In [5]:
# Looks like there was some minor trimming, but not substantial.

In [5]:
!qiime dada2 denoise-paired --help

Usage: qiime dada2 denoise-paired [OPTIONS]

  This method denoises paired-end sequences, dereplicates them, and filters
  chimeras.

Inputs:
  --i-demultiplexed-seqs ARTIFACT SampleData[PairedEndSequencesWithQuality]
                         The paired-end demultiplexed sequences to be
                         denoised.                                  [required]
Parameters:
  --p-trunc-len-f INTEGER
                         Position at which forward read sequences should be
                         truncated due to decrease in quality. This truncates
                         the 3' end of the of the input sequences, which will
                         be the bases that were sequenced in the last cycles.
                         Reads that are shorter than this value will be
                         discarded. After this parameter is applied there must
                         still be at least a 12 nucleotide overlap between the
                         forward and reverse reads. If 

In [8]:
# Pick OTUs for 2019 dataset
# Goal for trimming is to keep similar levels of quality to 2015 trimmed data
# Trimming at last instance of lower quartile above 20
# Error profiles are actually relatively similar at this point between the two years

!qiime dada2 denoise-paired \
  --p-n-threads 18 \
  --i-demultiplexed-seqs ../Seq-processing/sequences.cutadapt.qza \
  --p-trim-left-f 0 \
  --p-trim-left-r 0 \
  --p-trunc-len-f 220 \
  --p-trunc-len-r 202 \
  --p-max-ee-f 8 \
  --p-max-ee-r 8 \
  --p-trunc-q 2 \
  --p-min-overlap 10 \
  --p-pooling-method 'pseudo' \
  --p-chimera-method 'pooled' \
  --p-min-fold-parent-over-abundance 1.0 \
  --p-allow-one-off False \
  --p-n-reads-learn 1000000 \
  --o-table ../Seq-processing/table.2019.noITSx.cutadapt.qza \
  --o-representative-sequences ../Seq-processing/rep-seqs.2019.noITSx.cutadapt.qza \
  --o-denoising-stats ../Seq-processing/denoising-stats.2019.noITSx.cutadapt.qza \
  --verbose

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada.R --input_directory /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpwuepzv4c/forward --input_directory_reverse /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpwuepzv4c/reverse --output_path /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpwuepzv4c/output.tsv.biom --output_track /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpwuepzv4c/track.tsv --filtered_directory /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpwuepzv4c/filt_f --filtered_directory_reverse /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpwuepzv4c/filt_r --truncation_length 220 --truncation_length_reverse 202 --trim_left 0 --trim_left_reverse 0 --max_expected_errors 8 --max_expected_errors_reverse 8 --truncation_quality_score 2 --min_overlap 10 --pooli

In [9]:
# Look at seq lengths after untrimmed OTU picking to help inform future trimming
# Save rep seqs visualization
# Similar to 2015 dataset
!qiime feature-table tabulate-seqs \
    --i-data ../Seq-processing/rep-seqs.2019.noITSx.cutadapt.qza\
    --o-visualization ../Seq-processing/rep-seqs.2019.noITSx.cutadapt.qzv

Saved Visualization to: ../Seq-processing/rep-seqs.2019.noITSx.cutadapt.qzv


In [10]:
# This degree of trimming resulted in large fractions of reads making it through
# 65-92%. We expect to be missing the longer-read OTUs, but I think it's necessary
# to trim more aggressively to make the two datasets more comparable.
# Max OTU seq length here would be just over 400bp.
!qiime metadata tabulate --m-input-file ../Seq-processing/denoising-stats.2019.noITSx.cutadapt.qza --o-visualization ../Seq-processing/denoising-stats.2019.noITSx.cutadapt.qzv

Saved Visualization to: ../Seq-processing/denoising-stats.2019.noITSx.cutadapt.qzv


In [16]:
# Classify our 2019 OTUs using the UNITE-recommended database and classifier
!qiime feature-classifier classify-sklearn \
  --i-classifier ../Seq-processing/unite.classifier.trimmed.qza \
  --i-reads ../Seq-processing/rep-seqs.2019.noITSx.cutadapt.qza \
  --o-classification ../Seq-processing/taxonomy.2019.noITSx.cutadapt.qza

Saved FeatureData[Taxonomy] to: ../Seq-processing/taxonomy.2019.noITSx.cutadapt.qza


#### 2015 dataset cutadapt no ITSx

In [6]:
# Since not running ITSx, do need to trim primers in case short amplicons got primers sequenced into at 5' end.

!qiime cutadapt trim-paired \
    --i-demultiplexed-sequences ../Seq-processing/sequences.2015.qza \
    --p-adapter-f AACTTTYRRCAAYGGATCWCT \
    --p-adapter-r AGCCTCCGCTTATTGATATGCTTAART \
    --p-error-rate 0.1 \
    --p-match-adapter-wildcards \
    --p-discard-untrimmed False \
    --p-cores 18 \
    --o-trimmed-sequences ../Seq-processing/sequences.2015.cutadapt.qza

Saved SampleData[PairedEndSequencesWithQuality] to: ../Seq-processing/sequences.2015.cutadapt.qza


In [7]:
# Inspect efects
! qiime demux summarize \
  --i-data ../Seq-processing/sequences.2015.cutadapt.qza \
  --o-visualization ../Seq-processing/sequences.2015.cutadapt.qzv

Saved Visualization to: ../Seq-processing/sequences.2015.cutadapt.qzv


In [12]:
# Pick OTUs for 2015 dataset
# Trimming to match 2019 dataset quality profiles
# Trimming at last instance of lower quartile above 20
# Error profiles are actually relatively similar at this point between the two years

!qiime dada2 denoise-paired \
  --p-n-threads 18 \
  --i-demultiplexed-seqs ../Seq-processing/sequences.2015.cutadapt.qza \
  --p-trim-left-f 0 \
  --p-trim-left-r 0 \
  --p-trunc-len-f 226 \
  --p-trunc-len-r 201 \
  --p-max-ee-f 8 \
  --p-max-ee-r 8 \
  --p-trunc-q 2 \
  --p-min-overlap 10 \
  --p-pooling-method 'pseudo' \
  --p-chimera-method 'pooled' \
  --p-min-fold-parent-over-abundance 1.0 \
  --p-allow-one-off False \
  --p-n-reads-learn 1000000 \
  --o-table ../Seq-processing/table.2015.noITSx.cutadapt.qza \
  --o-representative-sequences ../Seq-processing/rep-seqs.2015.noITSx.cutadapt.qza \
  --o-denoising-stats ../Seq-processing/denoising-stats.2015.noITSx.cutadapt.qza \
  --verbose

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada.R --input_directory /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpm1ynvjrt/forward --input_directory_reverse /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpm1ynvjrt/reverse --output_path /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpm1ynvjrt/output.tsv.biom --output_track /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpm1ynvjrt/track.tsv --filtered_directory /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpm1ynvjrt/filt_f --filtered_directory_reverse /var/folders/fg/swg7hms154x088v8f_71v1500000gn/T/tmpm1ynvjrt/filt_r --truncation_length 226 --truncation_length_reverse 201 --trim_left 0 --trim_left_reverse 0 --max_expected_errors 8 --max_expected_errors_reverse 8 --truncation_quality_score 2 --min_overlap 10 --pooli

In [13]:
# 50-87% of 2015 reads getting through to end
# Might be somewhat lower than 2019 dataset, but not bad.
!qiime metadata tabulate --m-input-file ../Seq-processing/denoising-stats.2015.noITSx.cutadapt.qza \
    --o-visualization ../Seq-processing/denoising-stats.2015.noITSx.cutadapt.qzv

Saved Visualization to: ../Seq-processing/denoising-stats.2015.noITSx.cutadapt.qzv


In [14]:
# Look at seq lengths after untrimmed OTU picking to help inform future trimming
# Save rep seqs visualization
# Similar to 2019 dataset
!qiime feature-table tabulate-seqs \
    --i-data ../Seq-processing/rep-seqs.2015.noITSx.cutadapt.qza\
    --o-visualization ../Seq-processing/rep-seqs.2015.noITSx.cutadapt.qzv

Saved Visualization to: ../Seq-processing/rep-seqs.2015.noITSx.cutadapt.qzv


In [15]:
# Classify our 2015 OTUs using the UNITE-recommended database and classifier
!qiime feature-classifier classify-sklearn \
  --i-classifier ../Seq-processing/unite.classifier.trimmed.qza \
  --i-reads ../Seq-processing/rep-seqs.2015.noITSx.cutadapt.qza \
  --o-classification ../Seq-processing/taxonomy.2015.noITSx.cutadapt.qza

Saved FeatureData[Taxonomy] to: ../Seq-processing/taxonomy.2015.noITSx.cutadapt.qza
